In [45]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import xml.etree.ElementTree as et 

print('pd.__version__', pd.__version__)
print('np.__version__', np.__version__)
print('sns.__version__', sns.__version__)

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',200)
cwd = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/'
cwd = '/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/'

pd.__version__ 1.2.4
np.__version__ 1.20.3
sns.__version__ 0.11.2


In [46]:
#df = pd.read_csv(os.path.join(os.getcwd(), '20210207_BAT2_PREsubmission.csv'), sep=',', index_col=0)

# filename = 'BAT2_batch1_20210325' # BATCH1 CS 10 compounds
# filename = 'BAT2_batch2_20210325' # BATCH2 CS 10 compounds
# filename = 'BAT2_batch3_20210325' # BATCH3 CS 10 compounds + pos control
# filename = 'BAT2_batch4_20210330' # BATCH4 CS 10 compounds + pos control
# filename = 'BAT2_batch5_20210330' # BATCH5 CS only compound 1771


# # all DS
# filename = '20221125_BAT2_DS_batch1' # BATCH1 DS all compounds

# all DM
filename = '20221125_BAT2_DM_batch1' # BATCH1 DM all compounds

df = pd.read_csv(os.path.join(cwd, 'Percentages_4PRISM', filename+'.csv'), index_col=0)

In [47]:
df[df.excluded == True]
# correct amount of data?

print(df.shape, 41*96*3*1) # 41 compounds * 96 well plates 3 replicates 1 condition

(11808, 53) 11808


# 24 hpf sheet

## Compound Information
- Compound_ID --> compound
- Vehicle --> always DMSO?

## Plate Map Information
- Plate_Map_Name --> plate_name without Replicate indicator
- Well_Position --> well_name
- Dose --> dose
- Dose_unit --> unit
- Is_Vehicle_Control --> Is_Vehicle_Control
- Is_Positive_Control --> Is_Positive_Control
- Is_Excluded --> always False (whether the well should be excluded before the experiment (e.g., blank wells))**

## Screen Plate Information
- Plate_Name --> plate_name
** 120 hpf
- Plate_Screen_Time_End --> group plate_name creation_date max**
** 24 hpf
- Plate_Screen_Time_End --> alwyas 9.00 AM **

- Well_Comment --> np.nan
- Keep_Well_Data --> should refer to mortality data (precipitation etc) here always True (in that sense well data refers to mortality and in case images)

## without Title
- Keep_Embryo_Data --> is always True! **if False the all data for that embryo will be discarded** Can never be np.nan
- Embryo_Type --> R-C: repeated chorion, R-DC: repeated dechorionated, S-C: single chorion, S-DC: single dechorionated
                {'CS':'S-C', 'CM':'R-C', 'SC':'S-C', 'MC':'R-C'
                 'DS':,'S-DC', 'DM':'R-DC', 'SD':,'S-DC', 'MD':'R-DC'}
                
                or easier: 
                    exposure == 'single_dose' vs 'multiple_dose' --> 'S-' vs 'R-'
                    chorion == 'chorion' vs 'dechorionated' --> 'C' vs 'DC'
- Mortality --> mortality 24 and 120, respectively  dead24 dead120

***
## this only 120hpf
- Axis: curvature of body axis --> bodycurvature
- Craniofacial: eye defects --> Craniofacial: eye defects -- OK
- Craniofacial: snout defects --> Craniofacial: snout defects -- OK
- Craniofacial: jaw defects --> Craniofacial: snout defects (same because we take that as one PT)
- Craniofacial: edema --> Craniofacial: edema -- OK
- Craniofacial: abnormal head size --> np.nan (not quantified)
- Percardial tissue: Edema --> Percardial tissue: Edema -- OK
- Yolk sac: Edema --> Yolk sac: Edema -- OK
- Trunk: short --> Trunk: short -- OK
- Trunk: long --> Trunk: long -- OK
- Pigment: abnormal --> Pigment: abnormal -- OK
- Pigment: decreased --> Pigment: decreased -- OK
- Pigment: absent coloration --> Pigment: absent coloration -- OK (is np.nan here)
- *more endpoints can be included (lab specific) [check columns to give them...]

necrosis, tailbending, notochorddefects, finabsence, scoliosis, 
fishoutline_dorsal_image_area, fishoutline_lateral_image_area, fishoutline_unit, eyes_dorsal_image_eye_down_length
, eyes_dorsal_image_eye_up_length, avg_eye_diameter, pigmentation_dorsal_image_area_fish_wo_eyes
, pigmentation_dorsal_image_area_pigmentation, pigmentation_dorsal_image_area_ratio, pigmentation_unit
, pigmentation_threshold, heart_lateral_image_area, heart_unit, bodylength_lateral_image_length, bodylength_unit
***


- Hours_post-embryonic --> 24 and 120, repectively
- Image taken --> 24 is np.nan, 120 True if image is there (if well_folder not np.nan), False if not


In [48]:
df['plate_name'].unique()

array(['20220426_1036_DM_R1', '20220524_1036_DM_R5',
       '20220801_1036_DM_R6', '20220326_1046_DM_R1',
       '20220328_1046_DM_R2', '20220329_1046_DM_R3',
       '20220614_1072_DM_R2', '20220730_1072_DM_R4',
       '20220801_1072_DM_R5', '20220426_1111_DM_R1',
       '20220524_1111_DM_R5', '20220806_1111_DM_R6',
       '20220326_1171_DM_R1', '20220328_1171_DM_R2',
       '20220329_1171_DM_R3', '20220613_1200_DM_R1',
       '20220614_1200_DM_R2', '20220727_1200_DM_R4',
       '20220730_1229_DM_R5', '20220801_1229_DM_R6',
       '20221008_1229_DM_R8', '20220530_1240_DM_R2',
       '20220531_1240_DM_R3', '20220725_1240_DM_R5',
       '20220528_1247_DM_R1', '20220530_1247_DM_R2',
       '20220531_1247_DM_R3', '20220402_1271_DM_R1',
       '20220404_1271_DM_R2', '20220810_1271_DM_R6',
       '20220528_1323_DM_R1', '20220531_1323_DM_R3',
       '20220806_1323_DM_R4', '20220426_1355_DM_R1',
       '20220524_1355_DM_R5', '20220806_1355_DM_R6',
       '20220607_1409_DM_R3', '20220730_1409_D

In [49]:
ordered_columns = ['Compound_ID', 'Vehicle', 'Plate_Map_Name', 'Well_Position', 'Dose', 'Dose_unit'
                   , 'Is_Vehicle_Control', 'Is_Positive_Control', 'Is_Excluded', 'Plate_Name'
                   , 'Plate_Screen_Time_End', 'Well_Comment'
                   , 'Keep_Well_Data', 'Keep_Embryo_Data', 'Embryo_Type'
                   , 'Mortality'
                   , 'Axis: curvature of body axis' # <-- required by BAT (120)
                   , 'Craniofacial: eye defects'
                   , 'Craniofacial: snout defects'
                   , 'Craniofacial: jaw defects'
                   , 'Craniofacial: edema'
                   , 'Craniofacial: abnormal head size'
                   , 'Percardial tissue: Edema'
                   , 'Yolk sac: Edema'
                   , 'Trunk: short'
                   , 'Trunk: long'
                   , 'Pigment: abnormal'
                   , 'Pigment: decreased'
                   , 'Pigment: absent coloration' # <-- required by BAT (120)
                   , 'necrosis', 'tailbending', 'notochorddefects', 'finabsence', 'scoliosis', 'unhatched120' # <-- lab specific
                   , 'fishoutline_dorsal_image_area', 'fishoutline_lateral_image_area', 'fishoutline_unit'
                   , 'eyes_dorsal_image_eye_down_length', 'eyes_dorsal_image_eye_up_length', 'avg_eye_diameter'
                   , 'pigmentation_dorsal_image_area_fish_wo_eyes', 'pigmentation_dorsal_image_area_pigmentation'
                   , 'pigmentation_dorsal_image_area_ratio', 'pigmentation_unit', 'pigmentation_threshold'
                   , 'heart_lateral_image_area', 'heart_unit'
                   , 'bodylength_lateral_image_length', 'bodylength_unit' # <-- lab specific
                   , 'well_folder', 'lateral_image', 'dorsal_image', 'excluded'
                   , 'Hours_post-embryonic', 'Image taken']

ordered_columns24 = ['Compound_ID', 'Vehicle', 'Plate_Map_Name', 'Well_Position', 'Dose', 'Dose_unit'
                   , 'Is_Vehicle_Control', 'Is_Positive_Control', 'Is_Excluded', 'Plate_Name'
                   , 'Plate_Screen_Time_End', 'Well_Comment'
                     , 'Keep_Well_Data', 'Keep_Embryo_Data', 'Embryo_Type'
                   , 'Mortality'
                   , 'Hours_post-embryonic', 'Image taken']



df_squash = df.copy()

df_squash['Vehicle'] = 'DMSO'
df_squash['Is_Excluded'] = False
df_squash['Plate_Map_Name'] = df_squash['plate_name'].str.split('_R').str[0]
#df_squash['Plate_Screen_Time_End'] = np.nan
df_squash['Well_Comment'] = np.nan


df_squash['Keep_Well_Data'] = True


# Keep_Embryo_Data always True
df_squash['Keep_Embryo_Data'] = True
#df_squash.loc[df_squash['excluded']==False, 'Keep_Embryo_Data'] = True
#df_squash.loc[df_squash['excluded']==True, 'Keep_Embryo_Data'] = True

df_squash['Craniofacial: jaw defects'] = df_squash['Craniofacial: snout defects']
df_squash['Craniofacial: abnormal head size'] = np.nan

df_squash['Mortality'] = np.nan
df_squash['Hours_post-embryonic'] = np.nan


df_squash['Image taken'] = np.nan
df_squash.loc[df_squash['dorsal_image'].isnull(), 'Image taken'] = False
df_squash.loc[~df_squash['dorsal_image'].isnull(), 'Image taken'] = True
#df_squash['Image taken'] = df_squash['Image taken'].apply(lambda x: True if x == np.nan else x)

etd = {'CS':'S-C', 'CM':'R-C', 'SC':'S-C', 'MC':'R-C', 'DS':'S-DC', 'DM':'R-DC', 'SD':'S-DC', 'MD':'R-DC'}
df_squash['Embryo_Type'] = df_squash['Plate_Map_Name'].apply(lambda x: etd[x.split('_')[-1]])


df_squash.rename(columns={'compound':'Compound_ID', 'well_name':'Well_Position', 'dose':'Dose'
                          , 'unit':'Dose_unit', 'plate_name':'Plate_Name', 'bodycurvature':'Axis: curvature of body axis'}, inplace=True)


df_squash.tail()  # drop creation date then
df_squash['Image taken'].unique()

array([False, True], dtype=object)

In [50]:
df_squash.head()
df_squash.loc[df_squash['Compound_ID'].isnull()]

,Plate_Name,Well_Position,chorion,exposure,Compound_ID,Dose,Dose_unit,Is_Vehicle_Control,Is_Positive_Control,dead24,dead120,unhatched120,creation_date,well_folder,lateral_image,dorsal_image,show2user,excluded,correction_done,Axis: curvature of body axis,snoutjawdefects,yolkedema,necrosis,tailbending,notochorddefects,craniofacialedema,finabsence,scoliosis,fishoutline_dorsal_image_area,fishoutline_lateral_image_area,fishoutline_unit,eyes_dorsal_image_eye_down_length,eyes_dorsal_image_eye_up_length,avg_eye_diameter,pigmentation_dorsal_image_area_fish_wo_eyes,pigmentation_dorsal_image_area_pigmentation,pigmentation_dorsal_image_area_ratio,pigmentation_unit,pigmentation_threshold,heart_lateral_image_area,heart_unit,bodylength_lateral_image_length,bodylength_unit,Craniofacial: eye defects,Craniofacial: snout defects,Craniofacial: edema,Yolk sac: Edema,Trunk: short,Trunk: long,Percardial tissue: Edema,Pigment: abnormal,Pigment: decreased,Pigment: absent coloration,Vehicle,Is_Excluded,Plate_Map_Name,Well_Comment,Keep_Well_Data,Keep_Embryo_Data,Craniofacial: jaw defects,Craniofacial: abnormal head size,Mortality,Hours_post-embryonic,Image taken,Embryo_Type


# plate end times 24 hpf and 120 hpf

in 2020-07-07T08:53:40.178643+02:00[Europe/Paris]
+02:00[Europe/Paris] denotes the offset from GMT/UTC

France is GMT/UTC + 1h during Standard Time
France is GMT/UTC + 2h during Daylight Saving Time

that has no impact here whatsoever. Just drop

*** two separate dfs for 24 and 120 now***

# reset pos controls - not full plates

where there is no compound set
- Is_Excluded set to True
- Keep_Embryo_Data to np.nan
- Keep_Well_Data to np.nan

and also 'Is_Vehicle_Control' , 'Is_Positive_Control', 'dead24', 'dead120', 'unhatched120', 'Vehicle' to np.nan

In [51]:
cols2nan = ['Keep_Embryo_Data', 'Keep_Well_Data', 'Is_Vehicle_Control', 'Is_Positive_Control'
            , 'dead24', 'dead120', 'unhatched120', 'Vehicle']
df_squash.loc[df_squash['Compound_ID'].isnull(), cols2nan] = np.nan
df_squash.loc[df_squash['Compound_ID'].isnull(), 'Is_Excluded'] = True
df_squash.head(20)

,Plate_Name,Well_Position,chorion,exposure,Compound_ID,Dose,Dose_unit,Is_Vehicle_Control,Is_Positive_Control,dead24,dead120,unhatched120,creation_date,well_folder,lateral_image,dorsal_image,show2user,excluded,correction_done,Axis: curvature of body axis,snoutjawdefects,yolkedema,necrosis,tailbending,notochorddefects,craniofacialedema,finabsence,scoliosis,fishoutline_dorsal_image_area,fishoutline_lateral_image_area,fishoutline_unit,eyes_dorsal_image_eye_down_length,eyes_dorsal_image_eye_up_length,avg_eye_diameter,pigmentation_dorsal_image_area_fish_wo_eyes,pigmentation_dorsal_image_area_pigmentation,pigmentation_dorsal_image_area_ratio,pigmentation_unit,pigmentation_threshold,heart_lateral_image_area,heart_unit,bodylength_lateral_image_length,bodylength_unit,Craniofacial: eye defects,Craniofacial: snout defects,Craniofacial: edema,Yolk sac: Edema,Trunk: short,Trunk: long,Percardial tissue: Edema,Pigment: abnormal,Pigment: decreased,Pigment: absent coloration,Vehicle,Is_Excluded,Plate_Map_Name,Well_Comment,Keep_Well_Data,Keep_Embryo_Data,Craniofacial: jaw defects,Craniofacial: abnormal head size,Mortality,Hours_post-embryonic,Image taken,Embryo_Type
0,20220426_1036_DM_R1,A01,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
1,20220426_1036_DM_R1,A02,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
2,20220426_1036_DM_R1,A03,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
3,20220426_1036_DM_R1,A04,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
4,20220426_1036_DM_R1,A05,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
5,20220426_1036_DM_R1,A06,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
6,20220426_1036_DM_R1,A07,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
7,20220426_1036_DM_R1,A08,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC
8,20220426_1036_DM_R1,A09,dechorionated,multiple_dose,1036,50.00,uM,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0

In [52]:
# 120 hpf 

df120 = df_squash.copy()

# readable datetime format
df120[['Datetime', 'Datetime_offset']] = df120.creation_date.str.split("+", expand=True)
df120['Datetime'] = pd.to_datetime(df120['Datetime'])

# max time by plate
g = df120.groupby('Plate_Name')['Datetime'].agg('max').to_frame()
g.reset_index(inplace=True)
g.rename(columns={'Datetime':'Plate_Screen_Time_End'}, inplace=True)


df120 = pd.merge(left=df120, right=g, left_on='Plate_Name', right_on='Plate_Name', how='outer')
df120.tail(20)

,Plate_Name,Well_Position,chorion,exposure,Compound_ID,Dose,Dose_unit,Is_Vehicle_Control,Is_Positive_Control,dead24,dead120,unhatched120,creation_date,well_folder,lateral_image,dorsal_image,show2user,excluded,correction_done,Axis: curvature of body axis,snoutjawdefects,yolkedema,necrosis,tailbending,notochorddefects,craniofacialedema,finabsence,scoliosis,fishoutline_dorsal_image_area,fishoutline_lateral_image_area,fishoutline_unit,eyes_dorsal_image_eye_down_length,eyes_dorsal_image_eye_up_length,avg_eye_diameter,pigmentation_dorsal_image_area_fish_wo_eyes,pigmentation_dorsal_image_area_pigmentation,pigmentation_dorsal_image_area_ratio,pigmentation_unit,pigmentation_threshold,heart_lateral_image_area,heart_unit,bodylength_lateral_image_length,bodylength_unit,Craniofacial: eye defects,Craniofacial: snout defects,Craniofacial: edema,Yolk sac: Edema,Trunk: short,Trunk: long,Percardial tissue: Edema,Pigment: abnormal,Pigment: decreased,Pigment: absent coloration,Vehicle,Is_Excluded,Plate_Map_Name,Well_Comment,Keep_Well_Data,Keep_Embryo_Data,Craniofacial: jaw defects,Craniofacial: abnormal head size,Mortality,Hours_post-embryonic,Image taken,Embryo_Type,Datetime,Datetime_offset,Plate_Screen_Time_End
11788,20220802_1989_DM_R4,G05,dechorionated,multiple_dose,1989,0.51,uM,0.0,0.0,0.0,0.0,0.0,2022-08-02T08:36:01.365961+02:00[Europe/Paris],Well_G05,20220802_1989_DM_R4_tail_W_G05_1_2.png,20220802_1989_DM_R4_tail_W_G05_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1145991.00,1488209.25,micron_square,384.02,388.24,386.130,960467.75,312966.50,0.33,micron_square,45.0,35937.00,micron_square,3982.06,micron,False,False,False,False,False,False,False,False,False,NaN,DMSO,False,20220802_1989_DM,NaN,1.0,1.0,False,NaN,NaN,NaN,True,R-DC,2022-08-02 08:36:01.365961,02:00[Europe/Paris],2022-08-02 09:20:57.614271
11789,20220802_1989_DM_R4,G06,dechorionated,multiple_dose,1989,0.51,uM,0.0,0.0,0.0,0.0,0.0,2022-08-02T08:42:20.491472+02:00[Europe/Paris],Well_G06,20220802_1989_DM_R4_tail_W_G06_1_2.png,20220802_1989_DM_R4_tail_W_G06_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1163384.75,1477198.25,micron_square,370.38,369.65,370.015,987390.25,346513.75,0.35,micron_square,45.0,33063.25,micron_square,3998.64,micron,False,False,False,False,False,False,False,False,False,NaN,DMSO,False,20220802_1989_DM,NaN,1.0,1.0,False,NaN,NaN,NaN,True,R-DC,2022-08-02 08:42:20.491472,02:00[Europe/Paris],2022-08-02 09:20:57.614271
11790,20220802_1989_DM_R4,G07,dechorionated,multiple_dose,1989,0.51,uM,0.0,0.0,0.0,0.0,0.0,2022-08-02T08:47:07.004137+02:00[Europe/Paris],Well_G07,20220802_1989_DM_R4_tail_W_G07_1_2.png,20220802_1989_DM_R4_tail_W_G07_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1212117.50,1502094.00,micron_square,379.23,375.64,377.435,1032160.25,383721.25,0.37,micron_square,45.0,36209.25,micron_square,4009.74,micron,False,False,False,False,False,False,False,False,False,NaN,DMSO,False,20220802_1989_DM,NaN,1.0,1.0,False,NaN,NaN,NaN,True,R-DC,2022-08-02 08:47:07.004137,02:00[Europe/Paris],2022-08-02 09:20:57.614271
11791,20220802_1989_DM_R4,G08,dechorionated,multiple_dose,1989,0.51,uM,0.0,0.0,0.0,0.0,0.0,2022-08-02T08:54:23.497146+02:00[Europe/Paris],Well_G08,20220802_1989_DM_R4_tail_W_G08_1_2.png,20220802_1989_DM_R4_tail_W_G08_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1072786.00,1392528.50,micron_square,345.51,352.71,349.110,916000.25,337529.50,0.37,micron_square,45.0,42108.00,micron_square,3899.64,micron,True,False,False,False,False,False,False,False,False,NaN,DMSO,False,20220802_1989_DM,NaN,1.0,1.0,False,NaN,NaN,NaN,True,R-DC,2022-08-02 08:54:23.497146,02:00[Europe/Paris],2022-08-02 09:20:57.614271
11792,20220802_1989_DM_R4,G09,dechorionated,multiple_dose,1989,0.51,uM,0.0,0.0,0.0,0.0,0.0,2022-08-02T08:59:59.115924+02:00[Europe/Paris],Well_G09,20220802_1989_DM_R4_tail_W_G09_1_2.png,20220802_1989_DM_R4_tail_W_G09_1_1.png,1,False,True,False,False,False,Fal

In [53]:
# 24 hpf
# mortality screen 24h - minus 4 days at 9.00

df24 = df120.copy()

df24['Plate_Screen_Time_End'] = df24['Plate_Screen_Time_End'] + pd.DateOffset(-4)
df24['Plate_Screen_Time_End'] = df24['Plate_Screen_Time_End'].apply(lambda x: x.replace(hour=9, minute=0, second=0, microsecond=0))
#
df24.head()

,Plate_Name,Well_Position,chorion,exposure,Compound_ID,Dose,Dose_unit,Is_Vehicle_Control,Is_Positive_Control,dead24,dead120,unhatched120,creation_date,well_folder,lateral_image,dorsal_image,show2user,excluded,correction_done,Axis: curvature of body axis,snoutjawdefects,yolkedema,necrosis,tailbending,notochorddefects,craniofacialedema,finabsence,scoliosis,fishoutline_dorsal_image_area,fishoutline_lateral_image_area,fishoutline_unit,eyes_dorsal_image_eye_down_length,eyes_dorsal_image_eye_up_length,avg_eye_diameter,pigmentation_dorsal_image_area_fish_wo_eyes,pigmentation_dorsal_image_area_pigmentation,pigmentation_dorsal_image_area_ratio,pigmentation_unit,pigmentation_threshold,heart_lateral_image_area,heart_unit,bodylength_lateral_image_length,bodylength_unit,Craniofacial: eye defects,Craniofacial: snout defects,Craniofacial: edema,Yolk sac: Edema,Trunk: short,Trunk: long,Percardial tissue: Edema,Pigment: abnormal,Pigment: decreased,Pigment: absent coloration,Vehicle,Is_Excluded,Plate_Map_Name,Well_Comment,Keep_Well_Data,Keep_Embryo_Data,Craniofacial: jaw defects,Craniofacial: abnormal head size,Mortality,Hours_post-embryonic,Image taken,Embryo_Type,Datetime,Datetime_offset,Plate_Screen_Time_End
0,20220426_1036_DM_R1,A01,dechorionated,multiple_dose,1036,50.0,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC,NaT,NaN,2022-04-22 09:00:00
1,20220426_1036_DM_R1,A02,dechorionated,multiple_dose,1036,50.0,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC,NaT,NaN,2022-04-22 09:00:00
2,20220426_1036_DM_R1,A03,dechorionated,multiple_dose,1036,50.0,uM,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC,NaT,NaN,2022-04-22 09:00:00
3,20220426_1036_DM_R1,A04,dechorionated,multiple_dose,1036,50.0,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC,NaT,NaN,2022-04-22 09:00:00
4,20220426_1036_DM_R1,A05,dechorionated,multiple_dose,1036,50.0,uM,0.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DMSO,False,20220426_1036_DM,NaN,1.0,1.0,NaN,NaN,NaN,NaN,False,R-DC,NaT,NaN,2022-04-22 09:00:00


# mortality

- Mortality --> mortality 24 and 120, respectively  dead24 dead120

In [54]:
df120['Mortality'] = df120['dead120']
df24['Mortality'] = df24['dead24']

# hours post embryonic and reset image taken at 24 hpf

In [55]:
df120['Hours_post-embryonic'] = 120
df24['Hours_post-embryonic'] = 24
df24['Image taken'] = np.nan
#df24['Keep_Embryo_Data'] = True

# bona fide columns in order

In [56]:
df120_c = df120[ordered_columns].copy()
df24_c = df24[ordered_columns24].copy()
empty_sheet = pd.DataFrame()

# True False NAN 0 1 Formatting

In [57]:
# eye pain

# T F
tf_dict = {1:True, 0:False, np.nan:'NA', 1.0:True, 0.0:False}

c1 = ['Is_Vehicle_Control', 'Is_Positive_Control', 'Is_Excluded', 'Keep_Well_Data', 'Keep_Embryo_Data'
    , 'Image taken', 'excluded']


# 0 1 NA
zero_one_dict = {True:1, False:0, np.nan:'NA'}
c2 = ['Mortality', 'Embryo_Type', 'Mortality',
       'Axis: curvature of body axis', 'Craniofacial: eye defects',
       'Craniofacial: snout defects', 'Craniofacial: jaw defects',
       'Craniofacial: edema', 'Craniofacial: abnormal head size',
       'Percardial tissue: Edema', 'Yolk sac: Edema', 'Trunk: short',
       'Trunk: long', 'Pigment: abnormal', 'Pigment: decreased',
       'Pigment: absent coloration', 'necrosis', 'tailbending',
       'notochorddefects', 'finabsence', 'scoliosis', 'unhatched120']

for col in c1: 
    #print(col)
    df120_c[col].replace(1,True, inplace=True, regex=True)
    df120_c[col].replace(0,False, inplace=True, regex=True)
    df120_c[col].replace(np.nan,'NA', inplace=True, regex=True)
    
    if col in df24_c.columns: 
        df24_c[col].replace(1,True, inplace=True, regex=True)
        df24_c[col].replace(0,False, inplace=True, regex=True)
        df24_c[col].replace(np.nan,'NA', inplace=True, regex=True)
    


for col in c2: 
    df120_c[col].replace(True,1, inplace=True)
    df120_c[col].replace(False,0, inplace=True)
    df120_c[col].replace('True',1, inplace=True)
    df120_c[col].replace('False',0, inplace=True)
    df120_c[col].replace(np.nan,'NA', inplace=True, regex=True)
    df120_c[col].replace('ND','NA', inplace=True, regex=True)
    
    if col in df24_c.columns: 
        df24_c[col].replace(True,1, inplace=True)
        df24_c[col].replace(False,0, inplace=True)
        df24_c[col].replace('True',1, inplace=True)
        df24_c[col].replace('False',0, inplace=True)
        df24_c[col].replace(np.nan,'NA', inplace=True, regex=True)
        df24_c[col].replace('ND','NA', inplace=True, regex=True)
                

for col in df24_c.columns: 
    if col != 'Plate_Screen_Time_End':
        df24_c[col] = df24_c[col].replace(np.nan,'NA', regex=True)
for col in df120_c.columns: 
    if col != 'Plate_Screen_Time_End':
        df120_c[col] = df120_c[col].replace(np.nan,'NA', regex=True)

for col in c1+c2: 
    if col in df120_c.columns:
        print(col, df120_c[col].unique())
    
for col in c1+c2: 
    if col in df24_c.columns:
        print(col, df24_c[col].unique())

Is_Vehicle_Control [False  True]
Is_Positive_Control [False  True]
Is_Excluded [False]
Keep_Well_Data [ True]
Keep_Embryo_Data [ True]
Image taken [False  True]
excluded ['NA' False True]
Mortality [1. 0.]
Embryo_Type ['R-DC']
Mortality [1. 0.]
Axis: curvature of body axis ['NA' 0.0 1.0]
Craniofacial: eye defects ['NA' 0.0 1.0]
Craniofacial: snout defects ['NA' 0.0 1.0]
Craniofacial: jaw defects ['NA' 0.0 1.0]
Craniofacial: edema ['NA' 0.0 1.0]
Craniofacial: abnormal head size ['NA']
Percardial tissue: Edema ['NA' 0.0 1.0]
Yolk sac: Edema ['NA' 0.0 1.0]
Trunk: short ['NA' 0.0 1.0]
Trunk: long ['NA' 0.0 1.0]
Pigment: abnormal ['NA' '0.0' 0 1]
Pigment: decreased ['NA' '0.0' 0 1]
Pigment: absent coloration ['NA']
necrosis ['NA' 0.0 1.0]
tailbending ['NA' 0.0 1.0]
notochorddefects ['NA' 0.0 1.0]
finabsence ['NA' 0.0 1.0]
scoliosis ['NA' 0.0 1.0]
unhatched120 [0.]
Is_Vehicle_Control [False  True]
Is_Positive_Control [False  True]
Is_Excluded [False]
Keep_Well_Data [ True]
Keep_Embryo_Data [

In [58]:
print(df120_c.shape, df24_c.shape)
df120_c.head()

(11808, 56) (11808, 18)


,Compound_ID,Vehicle,Plate_Map_Name,Well_Position,Dose,Dose_unit,Is_Vehicle_Control,Is_Positive_Control,Is_Excluded,Plate_Name,Plate_Screen_Time_End,Well_Comment,Keep_Well_Data,Keep_Embryo_Data,Embryo_Type,Mortality,Axis: curvature of body axis,Craniofacial: eye defects,Craniofacial: snout defects,Craniofacial: jaw defects,Craniofacial: edema,Craniofacial: abnormal head size,Percardial tissue: Edema,Yolk sac: Edema,Trunk: short,Trunk: long,Pigment: abnormal,Pigment: decreased,Pigment: absent coloration,necrosis,tailbending,notochorddefects,finabsence,scoliosis,unhatched120,fishoutline_dorsal_image_area,fishoutline_lateral_image_area,fishoutline_unit,eyes_dorsal_image_eye_down_length,eyes_dorsal_image_eye_up_length,avg_eye_diameter,pigmentation_dorsal_image_area_fish_wo_eyes,pigmentation_dorsal_image_area_pigmentation,pigmentation_dorsal_image_area_ratio,pigmentation_unit,pigmentation_threshold,heart_lateral_image_area,heart_unit,bodylength_lateral_image_length,bodylength_unit,well_folder,lateral_image,dorsal_image,excluded,Hours_post-embryonic,Image taken
0,1036,DMSO,20220426_1036_DM,A01,50.0,uM,False,False,False,20220426_1036_DM_R1,2022-04-26 12:52:25.499638,NA,True,True,R-DC,1.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,120,False
1,1036,DMSO,20220426_1036_DM,A02,50.0,uM,False,False,False,20220426_1036_DM_R1,2022-04-26 12:52:25.499638,NA,True,True,R-DC,1.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,120,False
2,1036,DMSO,20220426_1036_DM,A03,50.0,uM,False,False,False,20220426_1036_DM_R1,2022-04-26 12:52:25.499638,NA,True,True,R-DC,0.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,120,False
3,1036,DMSO,20220426_1036_DM,A04,50.0,uM,False,False,False,20220426_1036_DM_R1,2022-04-26 12:52:25.499638,NA,True,True,R-DC,1.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,120,False
4,1036,DMSO,20220426_1036_DM,A05,50.0,uM,False,False,False,20220426_1036_DM_R1,2022-04-26 12:52:25.499638,NA,True,True,R-DC,1.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,120,False


# SAVE as Excel

In [59]:
filename = '20221125_BAT2_DM_batch1'
print(os.path.join(cwd, 'Percentages_4PRISM', filename+'.xlsx'))
with pd.ExcelWriter(os.path.join(cwd, 'Percentages_4PRISM', filename+'.xlsx')) as writer: 
    empty_sheet.to_excel(writer, sheet_name='notes', index=False)
    df24_c.to_excel(writer, sheet_name='24_hpf', index=False, startrow=1)
    df120_c.to_excel(writer, sheet_name='120_hpf', index=False, startrow=1)

/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/Percentages_4PRISM/20221125_BAT2_DM_batch1.xlsx


In [44]:
df120_c[df120_c['Compound_ID']=='1547'][['Plate_Screen_Time_End', 'Plate_Map_Name']].drop_duplicates()
#df24_c[df24_c['Compound_ID']=='1547'][['Plate_Screen_Time_End', 'Plate_Map_Name']].drop_duplicates()

,Plate_Screen_Time_End,Plate_Map_Name
4608,2022-05-23 09:54:30.079568,20220523_1547_DS
4704,2022-02-07 11:03:47.735384,20220207_1547_DS
4800,2022-02-14 16:14:26.355847,20220214_1547_DS


# TODO: then add manually row 1 with merged cells and content of notes sheet

### take it from Phase 2_Interlab study data_suggested_format.xlsx

In [60]:
print(df24_c.columns)
print(df120_c.columns)
for item in [df24_c, df120_c]:
    print('Keep_Embryo_Data', item['Keep_Embryo_Data'].unique())
    print('Compound_ID', item['Compound_ID'].unique())
    print('Plate_Map_Name', item['Plate_Map_Name'].unique())

Index(['Compound_ID', 'Vehicle', 'Plate_Map_Name', 'Well_Position', 'Dose',
       'Dose_unit', 'Is_Vehicle_Control', 'Is_Positive_Control', 'Is_Excluded',
       'Plate_Name', 'Plate_Screen_Time_End', 'Well_Comment', 'Keep_Well_Data',
       'Keep_Embryo_Data', 'Embryo_Type', 'Mortality', 'Hours_post-embryonic',
       'Image taken'],
      dtype='object')
Index(['Compound_ID', 'Vehicle', 'Plate_Map_Name', 'Well_Position', 'Dose',
       'Dose_unit', 'Is_Vehicle_Control', 'Is_Positive_Control', 'Is_Excluded',
       'Plate_Name', 'Plate_Screen_Time_End', 'Well_Comment', 'Keep_Well_Data',
       'Keep_Embryo_Data', 'Embryo_Type', 'Mortality',
       'Axis: curvature of body axis', 'Craniofacial: eye defects',
       'Craniofacial: snout defects', 'Craniofacial: jaw defects',
       'Craniofacial: edema', 'Craniofacial: abnormal head size',
       'Percardial tissue: Edema', 'Yolk sac: Edema', 'Trunk: short',
       'Trunk: long', 'Pigment: abnormal', 'Pigment: decreased',
       'Pigme

# some more sanity checks

** eventually include that in the BAT_process**

for each compound-concentration: 

- if there is none dead: then we need to bring 7 pictures. 
- if there's at least one dead we do not need to bring 7 pictures

- we need a 20% of max mortality in the negative ctrl; maximum 2 of 11 dead in DMSO
- and at least a 20% in the positive (1 out of 8); minimum 1 of 8 dead in DCA

In [ ]:
df2sanity_check = df120_c[(df120_c['Plate_Name']=='20220208_1036_DS_R1') & 
    (df120_c['Compound_ID']=='DCA')]
df2sanity_check.shape

In [ ]:
df2sanity_check = df120_c.copy()
# df120_c.groupby(['Compound_ID', 'Plate_Name'])['excluded'].count()
df2sanity_check[['Compound_ID', 'Plate_Map_Name', 'Well_Position', 'Dose',
       'Dose_unit', 
       'Plate_Name', 'Plate_Screen_Time_End', 'Mortality',
       'well_folder',
       'lateral_image', 'dorsal_image', 'excluded', 'Hours_post-embryonic',
       'Image taken']].head()

In [ ]:
df2sanity_check['excluded'].replace("NA", np.nan, regex=True, inplace=True)
df2sanity_check['bodylength_lateral_image_length'].replace("NA", np.nan, regex=True, inplace=True)

df2sanity_check['excluded'] = pd.to_numeric(df2sanity_check['excluded'])
df2sanity_check['bodylength_lateral_image_length'] = pd.to_numeric(df2sanity_check['bodylength_lateral_image_length'])
df2sanity_check.dtypes

In [ ]:
g = df2sanity_check.groupby(['Compound_ID', 'Dose', 'Plate_Name'])[['Well_Position', 'Image taken', 'excluded']].sum()
print(g)

In [ ]:
df2sanity_check_oc = df2sanity_check[~df2sanity_check['Compound_ID'].isin(['DMSO', 'DCA'])].copy()
g = df2sanity_check_oc.groupby(['Compound_ID', 'Dose', 'Plate_Name'])[['bodylength_lateral_image_length', 'excluded']].count()
g.to_csv(os.path.join(cwd, 'tmp.csv'))
print(g)
